In [1]:
# from pyiron_contrib.workflow import nodes  # TODO: Update pre-built nodes
from pyiron_contrib.workflow.node import Node
from pyiron_contrib.workflow.workflow import Workflow

/Users/huber/work/pyiron/pyiron_contrib/pyiron_contrib/__init__.py:9: UserWarning: pyiron module not found, importing Project from pyiron_base
  warnings.warn("pyiron module not found, importing Project from pyiron_base")


# Pyiron workflows: Introduction and Syntax

Here we will highlight:
- The two (and a half) ways of defining a node
- The two x two ways of setting node channel values/connections
- The four ways of adding nodes to a workflow

Then we will present a simple working example (TODO)

## Defining a node

Nodes can be defined on-the-fly by passing any callable to the `Node` class, along with a string (tuple of strings) giving names for the output value(s).

In [2]:
def plus_minus_one(x):
    return x+1, x-1

try:
    node = Node(plus_minus_one, ("p1", "m1"))
except TypeError:
    print("This won't quite work yet!")

This won't quite work yet!


Unless you specify additional kwargs (i.e. `run_automatically=False`, `update_on_instantiation=False`), your node will try to execute the function immediately! In the example above, this resulted in a `TypeError` (from `None + 1`).

Instead of just preventing the update by decree, we could (a) provide default values for our function, and/or (b) provide type hints for our function (the node will only run automatically once all the input data has the right type (where specified)).
Let's do both

In [3]:
def plus_minus_one(x: int | float = 1) -> tuple[int | float, int | float]:
    return x+1, x-1

node = Node(plus_minus_one, ("p1", "m1"))

When we create a node, input and output data channels are constructed automatically from the signature and type hints of the function, and the output labels.

In [4]:
print(node.inputs.labels, node.outputs.labels)

['x'] ['p1', 'm1']


In [5]:
print(node.inputs.x.type_hint)

int | float


And now that it could safely update right away, we can look at the output:

In [6]:
print(node.outputs.to_value_dict())

{'p1': 2, 'm1': 0}


We can assign new data to the input channels with the `update` method, or with `=`:

In [7]:
node.inputs.x.update(2)
print(node.outputs.to_value_dict())

{'p1': 3, 'm1': 1}


In [8]:
node.inputs.x = 3
print(node.outputs.to_value_dict())

{'p1': 4, 'm1': 2}


If we're going to use a node many times, we may want to define a new sub-class of `Node` to handle this

In [9]:
from functools import partialmethod

class Adder(Node):
    @staticmethod
    def adder(x: int|float, y: int|float) -> int|float:
        return x + y
    
    __init__ = partialmethod(
        Node.__init__,
        node_function=adder,
        output_labels="sum"
    )

We could also have done this without `partialmethod` by simply overriding `__init__`, but this way is much tidier.

Note that we only need a single string for `output_labels` when we only return a single value, and that the node's `label` takes (by default) the name of the `node_function`:

In [10]:
add = Adder(x=2, y=3)
print(add.label)
print(add.outputs.sum)

adder
5


Here we have also shown another way to initialize data -- right at instantiation as a kwarg.
Kwarg values matching the node function signature will be passed from the node initializer to the channels.

Next, we want to make connections between nodes. Instead of `update` we can use the `connect` method to accomplish this, or we can do it with the other syntactic sugar we saw for regular data:

In [11]:
upstream = Adder(x=0, y=0)

ds1 = Node(plus_minus_one, ("p1", "m1"))
ds1.inputs.x.connect(upstream.outputs.sum)

ds2 = Node(plus_minus_one, ("p1", "m1"))
ds2.inputs.x = upstream.outputs.sum

ds3 = Node(plus_minus_one, ("p1", "m1"), x=upstream.outputs.sum)

print(ds1.outputs.p1, ds2.outputs.p1, ds3.outputs.p1)
upstream.inputs.x = 10
print(ds1.outputs.p1, ds2.outputs.p1, ds3.outputs.p1)

2 2 2
11 11 11


## Adding nodes to a workflow

We use workflows to hold together collections of related and connected nodes.

All five of the following approaches are equivalent ways to add a node to a workflow:

In [12]:
n1 = Node(plus_minus_one, ("p1", "m1"), label="n1")

wf = Workflow("my_wf", n1)  # As args at init
wf.add.Node(plus_minus_one, ("p1", "m1"), label="n2")  # Instantiating from the node adder
wf.add(Node(plus_minus_one, ("p1", "m1"), label="n3"))  # Instantiating then passing to node adder
wf.n4 = Node(plus_minus_one, ("p1", "m1"), label="whatever_n4_gets_used")  # By attribute
Node(plus_minus_one, ("p1", "m1"), label="n5", workflow=wf)  # By passing the workflow

wf.nodes

/Users/huber/work/pyiron/pyiron_contrib/pyiron_contrib/workflow/workflow.py:137: UserWarning: Reassigning the node whatever_n4_gets_used to the label n4 when adding it to the workflow my_wf.
  warn(


{'n1': <pyiron_contrib.workflow.node.Node at 0x10ae7b950>,
 'n2': <pyiron_contrib.workflow.node.Node at 0x14a29ae90>,
 'n3': <pyiron_contrib.workflow.node.Node at 0x14a29b590>,
 'n4': <pyiron_contrib.workflow.node.Node at 0x14a29bdd0>,
 'n5': <pyiron_contrib.workflow.node.Node at 0x14a290a10>}